# Nettoyage de tweets pour LLM

L'objectif de ce notebook est de préparer un corpus de tweets pour un usage avec un modèle de langage (LLM).

On va :

1. Charger un fichier CSV d'export Twitter.
2. Harmoniser les noms de colonnes (id, texte, etc.).
3. Séparer les tweets :
   - des **comptes Free** (annonces, réponses, retweets de Free),
   - des **clients** (utilisateurs finaux).
4. Nettoyer le texte des tweets clients :
   - suppression des URLs ;
   - normalisation des sauts de ligne (`\n`, `\\n`, `\r`, `\t`) en espaces ;
   - conservation des hashtags ;
   - passage en minuscules.
5. Dédupliquer les tweets sur l’identifiant `id`.
6. Produire des statistiques de contrôle (nombre de tweets, retweets Free, hashtags perdus…).

À la fin, on obtient un fichier CSV propre contenant uniquement des tweets clients nettoyés, prêt pour le LLM.


In [ ]:
import re
import html
import pandas as pd
from pathlib import Path

## 1. Fonctions utilitaires (regex, normalisation, hashtags, noms de colonnes)

Dans cette partie, on définit :

- des **expressions régulières** pour détecter :
  - les URLs ;
  - les retweets (tweets commençant par `RT`).
- une fonction `normalize_whitespace_tokens` qui transforme toutes les variantes de sauts de ligne et tabulations (`\n`, `\\n`, `\r`, `\\r`, `\t`, `\\t`) en espaces ;
- une fonction `extract_hashtags` qui extrait la liste des hashtags à partir d'un texte **déjà normalisé** ;
- une fonction `_normalize_col_name` qui homogénéise les noms de colonnes (minuscules, underscores, suppression de caractères cachés).

In [ ]:
# Regex globales

URL_PAT = re.compile(r'https?://\S+|www\.\S+', flags=re.IGNORECASE)
RT_PAT  = re.compile(r'^\s*rt\b', flags=re.IGNORECASE)  # RT au début de texte

# Utils texte

def normalize_whitespace_tokens(s: str) -> str:
    """
    Normalise les sauts de ligne / tabulations :
    - remplace les séquences texte '\\n', '\\r', '\\t'
    - ET les vrais caractères '\n', '\r', '\t'
    par des espaces.
    """
    if not isinstance(s, str):
        return s

    # Séquences échappées (backslash + lettre)
    s = s.replace("\\r", " ").replace("\\n", " ").replace("\\t", " ")
    # Caractères réels de contrôle
    s = s.replace("\r", " ").replace("\n", " ").replace("\t", " ")
    return s


def extract_hashtags(s_norm: str):
    """
    Extrait les hashtags d'une chaîne DEJA normalisée
    (pas de \\n / \\t bizarres).
    Retourne une liste de tags commençant par '#'.
    """
    if not isinstance(s_norm, str):
        return []
    return re.findall(r'#[^\s#]+', s_norm, flags=re.UNICODE)


def _normalize_col_name(c) -> str:
    """
    Normalise les noms de colonnes :
    - supprime le BOM éventuel
    - strip
    - lower
    - remplace les espaces par des underscores
    """
    return (
        str(c)
        .replace('\ufeff', '')  # BOM éventuel
        .strip()
        .lower()
        .replace(" ", "_")
    )


## 2. Fonction de nettoyage du texte pour le LLM

La fonction `clean_llm_text` prend un texte **déjà normalisé** (sans `\n` bizarres) et :

1. Décode les entités HTML (`&amp;`, `&quot;`, etc.).
2. Supprime les URLs.
3. Normalise les espaces (un seul espace entre les mots).
4. Vérifie que les hashtags présents dans le texte normalisé initial n'ont pas été perdus
   et réinjecte ceux qui manquent à la fin du texte.
5. Passe tout en minuscules (`casefold`).

Le résultat est stocké dans la colonne `text_clean_llm`.


In [ ]:
def clean_llm_text(text_norm: str) -> str:
    """
    Nettoie un texte de tweet DEJA normalisé (text_norm) pour usage LLM :
    1. Décodage HTML (&amp;, &quot;, etc.).
    2. Suppression des URLs.
    3. Normalisation des espaces.
    4. Vérification / réinjection des hashtags perdus.
    5. Passage en minuscules (casefold).
    """
    if not isinstance(text_norm, str):
        text_norm = str(text_norm) if text_norm is not None else ""

    # Décodage HTML (&amp; -> &, etc.)
    cleaned = html.unescape(text_norm)

    # Suppression des URLs
    cleaned = URL_PAT.sub(' ', cleaned)

    # Normalisation des espaces (un seul espace entre les mots)
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()

    # Gestion des hashtags possiblement perdus
    raw_tags     = extract_hashtags(text_norm)   # hashtags sur texte brut normalisé
    cleaned_tags = extract_hashtags(cleaned)     # hashtags sur texte nettoyé

    cleaned_lower = {t.casefold() for t in cleaned_tags}
    missing = [t for t in raw_tags if t.casefold() not in cleaned_lower]

    if missing:
        if cleaned and not cleaned.endswith(' '):
            cleaned += ' '
        cleaned += ' '.join(missing)
        cleaned = re.sub(r'\s+', ' ', cleaned).strip()

    # Minuscule robuste
    return cleaned.casefold()


## 3. Lecture robuste du fichier CSV

La fonction `read_csv_robust` permet de charger le fichier CSV d'export Twitter :

- essaie d'abord le format `UTF-8` ;
- en cas d'échec, réessaie en `UTF-8-SIG` (fréquent pour les exports Excel) ;
- laisse `pandas` détecter automatiquement le séparateur ;
- force toutes les colonnes en chaînes (`dtype=str`).

On obtient un `DataFrame` brut qui sera ensuite nettoyé.


In [ ]:
def read_csv_robust(path: Path) -> pd.DataFrame:
    """
    Essaie UTF-8 puis UTF-8-SIG (Excel) avec détection automatique du séparateur.
    Tout est lu en string (dtype=str).
    """
    try:
        return pd.read_csv(
            path,
            sep=None,
            engine="python",
            encoding="utf-8",
            on_bad_lines="skip",
            dtype=str,
        )
    except Exception:
        return pd.read_csv(
            path,
            sep=None,
            engine="python",
            encoding="utf-8-sig",
            on_bad_lines="skip",
            dtype=str,
        )


## 4. Pipeline global de nettoyage des tweets

La fonction `clean_dataframe(df)` applique tout le pipeline :

1. Normalisation des noms de colonnes (minuscules, underscore, suppression BOM).
2. Vérification de la présence de la colonne `id` (utilisée pour dédupliquer).
3. Détection de la colonne texte (`full_text`, `text`, `tweet` ou `content`) et copie dans `text_raw`.
4. Normalisation de `text_raw` avec `normalize_whitespace_tokens` (tous les `\n`, `\\n`, `\r`, `\\r`, `\t`, `\\t` → espaces).
5. Détection des retweets sur tout le dataset (`is_retweet`) :
   - texte commençant par `RT` ;
   - ou `retweeted_status` non nul si la colonne existe.
6. Filtre des comptes Free (`free`, `free_1337`, `freebox`, `freefoot`, `freenewsactu`, `universfreebox`) :
   - comptage des tweets Free filtrés (`reste`) ;
   - comptage des retweets Free (`retweets_free`) ;
   - conservation uniquement des tweets **clients** dans `df`.
7. Extraction des hashtags bruts des tweets clients → colonne `hashtags_list`.
8. Nettoyage du texte client pour le LLM → colonne `text_clean_llm`.
9. Contrôle des hashtags perdus → colonne `lost_hashtags_count`.
10. Déduplication sur `id` (un tweet client unique par id).
11. Calcul des statistiques globales (`rows_in`, `rows_out`, `retweets_free`, `retweets_total`, `reste`, `dedup_removed`, `lost_hashtags_total`).

La fonction renvoie :
- `df` : DataFrame nettoyé (tweets clients),
- `stats` : dictionnaire de statistiques de nettoyage.


In [ ]:
def clean_dataframe(df: pd.DataFrame):
    """
    Pipeline complet de nettoyage des tweets.
    """
    orig_len = len(df)

    # Normalisation des noms de colonnes (avec gestion BOM)
    df = df.copy()
    df.columns = [_normalize_col_name(c) for c in df.columns]

    # Vérification de la colonne 'id'
    if "id" not in df.columns:
        raise SystemExit(
            "La colonne 'id' est obligatoire pour dédupliquer les tweets.\n"
            f"Colonnes disponibles après normalisation : {', '.join(df.columns)}"
        )

    # Détection de la colonne texte
    text_col = next((c for c in ["full_text", "text", "tweet", "content"] if c in df.columns), None)
    if not text_col:
        raise SystemExit("Colonne texte introuvable (full_text / text / tweet / content).")

    # Texte brut -> text_raw
    df["text_raw"] = df[text_col].astype(str)

    # Normaliser UNE FOIS text_raw (il devient notre texte de référence "propre")
    df["text_raw"] = df["text_raw"].apply(normalize_whitespace_tokens)

    # Détection des retweets sur TOUT le dataset (avant filtre Free)
    has_retweeted_status = "retweeted_status" in df.columns
    if has_retweeted_status:
        retweeted_status_notna = df["retweeted_status"].notna()
    else:
        retweeted_status_notna = False

    df["is_retweet"] = df["text_raw"].str.match(RT_PAT, na=False) | retweeted_status_notna

    # Filtre des comptes Free (annonces / réponses / retweets de Free)
    POSSIBLE_HANDLE_COLS = [
        "screen_name", "user_screen_name", "username",
        "user", "author", "account", "handle"
    ]
    handle_col = next((c for c in POSSIBLE_HANDLE_COLS if c in df.columns), None)
    removed = 0
    retweets_free = 0  # retweets dans les lignes filtrées (Free)

    if handle_col:
        df["__handle_norm"] = (
            df[handle_col].astype(str)
            .str.replace(r"^@", "", regex=True)
            .str.strip()
            .str.casefold()
        )
        BLOCKED = {"free", "free_1337", "freebox", "freefoot", "freenewsactu", "universfreebox"}
        mask_free = df["__handle_norm"].isin(BLOCKED)

        removed = int(mask_free.sum())
        # Retweets parmi les tweets Free (avant de les enlever)
        retweets_free = int(df.loc[mask_free, "is_retweet"].sum())

        # On enlève les lignes Free pour ne garder que les tweets client
        df = df[~mask_free].drop(columns=["__handle_norm"])
    else:
        mask_free = None  # cohérence

    # Hashtags bruts sur les tweets clients (à partir de text_raw déjà normalisé)
    df["hashtags_list"] = df["text_raw"].apply(extract_hashtags)

    # Texte nettoyé LLM à partir de text_raw (déjà normalisé)
    df["text_clean_llm"] = df["text_raw"].apply(clean_llm_text)

    # Contrôle des hashtags perdus
    def _lost_hashtags_count_row(row):
        raw_list = row.get("hashtags_list")
        if not isinstance(raw_list, list):
            raw_list = []
        rset = {t.casefold() for t in raw_list}
        cset = {t.casefold() for t in extract_hashtags(row.get("text_clean_llm", ""))}
        return max(0, len(rset - cset))

    df["lost_hashtags_count"] = df.apply(_lost_hashtags_count_row, axis=1)

    # Déduplication uniquement sur id
    before = len(df)
    df["id"] = df["id"].astype(str).str.strip()
    df = df.dropna(subset=["id"]).drop_duplicates(subset=["id"], keep="first")
    dedup_removed = before - len(df)

    # Stats
    rows_out = int(len(df))
    retweets_client = int(df["is_retweet"].sum())  # côté clients (non affiché)
    retweets_total = retweets_client + retweets_free
    reste = int(removed)  # tweets Free filtrés

    stats = {
        "rows_in": int(orig_len),
        "rows_out": rows_out,
        "tweets_client": rows_out,
        "retweets_free": retweets_free,
        "retweets_total": retweets_total,
        "reste": reste,
        "dedup_removed": int(dedup_removed),
        "filtered_handles": int(removed),
        "lost_hashtags_total": int(df["lost_hashtags_count"].sum()),
    }
    return df, stats
